In [71]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [72]:
df=sns.load_dataset("diamonds")

In [73]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [74]:
df=df.select_dtypes(np.number)  # sadece sayısal değer içeren feature'ları aldık

In [75]:
df.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


# Isolation Forest ile Anomali Tespiti

In [4]:
from pyod.models.iforest import IForest

In [26]:
iforest=IForest()

In [27]:
iforest_model=iforest.fit(df)

In [211]:
iforest_model.labels_[:20] 

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0])

In [28]:
outliers=df[iforest_model.labels_==1]

In [29]:
outliers

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
5,0.24,62.8,57.0,336,3.94,3.96,2.48
6,0.24,62.3,57.0,336,3.95,3.98,2.47
...,...,...,...,...,...,...,...
53756,0.84,55.6,64.0,2724,6.42,6.32,3.54
53757,0.72,55.4,64.0,2724,6.06,5.97,3.34
53785,0.89,64.3,65.0,2728,6.00,5.95,3.84
53800,0.90,68.7,62.0,2732,5.83,5.79,3.99


In [30]:
len(df)

53940

### cutoff değeri kullanarak maskeleme yapıyoruz

In [31]:
probs=iforest_model.predict_proba(outliers)

In [32]:
probs[:10]

array([[0.43168113, 0.56831887],
       [0.34570679, 0.65429321],
       [0.21285381, 0.78714619],
       [0.49837282, 0.50162718],
       [0.5036481 , 0.4963519 ],
       [0.57206987, 0.42793013],
       [0.31536163, 0.68463837],
       [0.3692724 , 0.6307276 ],
       [0.44697906, 0.55302094],
       [0.37938351, 0.62061649]])

In [33]:
probs[:,1]

array([0.56831887, 0.65429321, 0.78714619, ..., 0.42261234, 0.42959899,
       0.38204605])

In [34]:
outliers[probs[:,1]>=0.65]

,carat,depth,table,price,x,y,z
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
8,0.22,65.1,61.0,337,3.87,3.78,2.49
14,0.20,60.2,62.0,345,3.79,3.75,2.27
54,0.22,59.3,62.0,404,3.91,3.88,2.31
...,...,...,...,...,...,...,...
47303,0.23,56.2,60.0,395,4.14,4.19,2.34
49556,0.71,64.1,60.0,2130,0.00,0.00,0.00
49557,0.71,64.1,60.0,2130,0.00,0.00,0.00
52860,0.50,79.0,73.0,2579,5.21,5.18,4.09


### yeni cutoff değeri ile belirlediğimiz outlier gözlemleri veri setinden çıkarıyoruz

In [35]:
dropped=outliers[probs[:,1]>=0.65].index

In [222]:
df.drop(dropped, 0, inplace=True)

In [223]:
df

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75
5,0.24,62.8,57.0,336,3.94,3.96,2.48
...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74


In [224]:
df.reset_index(inplace=True, drop=False)  # index numaralarını düzelttik

In [225]:
df

,index,carat,depth,table,price,x,y,z
0,0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,3,0.29,62.4,58.0,334,4.20,4.23,2.63
3,4,0.31,63.3,58.0,335,4.34,4.35,2.75
4,5,0.24,62.8,57.0,336,3.94,3.96,2.48
...,...,...,...,...,...,...,...,...
53235,53935,0.72,60.8,57.0,2757,5.75,5.76,3.50
53236,53936,0.72,63.1,55.0,2757,5.69,5.75,3.61
53237,53937,0.70,62.8,60.0,2757,5.66,5.68,3.56
53238,53938,0.86,61.0,58.0,2757,6.15,6.12,3.74


# Hyperparameter tuning

In [76]:
df

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74


In [11]:
contamination=[0.01, 0.05, 0.1, 0.2, 0.3]
n_estimators=[100, 300, 500]
max_samples=[0.5, 0.6]
max_features=[0.7, 0.9]

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from category_encoders import TargetEncoder

In [10]:
X=df.drop("price",1)
y=df["price"]

In [12]:
def iforest_rmse(X, y, contamination, n_estimators, max_samples, max_features):
    
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=10)
    
    
    variations=[ [a,b,c,d] for a in contamination for b in n_estimators for c in max_samples for d in max_features]
    
    
    for j,k,l,i in variations:
        iforest=IForest(contamination=j, n_estimators=k, max_samples=l, max_features=i, n_jobs=-1)
        iforest_model=iforest.fit(X_train)
        X_train_new=X_train[iforest_model.labels_==0]
        y_train_new=y_train.loc[X_train_new.index.to_list()]
        
        lr=LinearRegression()
        lr.fit(X_train_new, y_train_new)
        pred=lr.predict(X_test)
        mse=mean_squared_error(y_test, pred)
        rmse=np.sqrt(mse)     
        
        print(f"{j,k,l,i} : {round(rmse,4)}")

In [15]:
iforest_rmse(X, y, contamination, n_estimators, max_samples, max_features)

(0.01, 100, 0.5, 0.7) : 2298.3008
(0.01, 100, 0.5, 0.9) : 2297.7768
(0.01, 100, 0.6, 0.7) : 2385.2734
(0.01, 100, 0.6, 0.9) : 2604.8999
(0.01, 300, 0.5, 0.7) : 2338.8617
(0.01, 300, 0.5, 0.9) : 2310.3241
(0.01, 300, 0.6, 0.7) : 2353.8149
(0.01, 300, 0.6, 0.9) : 2392.6331
(0.01, 500, 0.5, 0.7) : 2347.2738
(0.01, 500, 0.5, 0.9) : 2330.9139
(0.01, 500, 0.6, 0.7) : 2350.9239
(0.01, 500, 0.6, 0.9) : 2443.8125
(0.05, 100, 0.5, 0.7) : 2420.837
(0.05, 100, 0.5, 0.9) : 2460.7591
(0.05, 100, 0.6, 0.7) : 2492.9637
(0.05, 100, 0.6, 0.9) : 2536.9004
(0.05, 300, 0.5, 0.7) : 2416.7513
(0.05, 300, 0.5, 0.9) : 2457.5033
(0.05, 300, 0.6, 0.7) : 2488.5312
(0.05, 300, 0.6, 0.9) : 2524.5214
(0.05, 500, 0.5, 0.7) : 2507.7966
(0.05, 500, 0.5, 0.9) : 2506.1268
(0.05, 500, 0.6, 0.7) : 2529.3128
(0.05, 500, 0.6, 0.9) : 2507.7165
(0.1, 100, 0.5, 0.7) : 2727.4423
(0.1, 100, 0.5, 0.9) : 2689.9856
(0.1, 100, 0.6, 0.7) : 2848.4683
(0.1, 100, 0.6, 0.9) : 2771.1877
(0.1, 300, 0.5, 0.7) : 2748.2861
(0.1, 300, 0.5, 0.9)

In [ ]:
# (0.01, 100, 0.5, 0.9) : 2297.7768

In [41]:
from pyod.models.iforest import IForest

In [42]:
iforest=IForest(contamination=0.01, n_estimators=100, max_samples=0.5, max_features=0.9)

In [43]:
iforest_model=iforest.fit(df)

In [44]:
outliers=df[iforest_model.labels_==1]

In [46]:
outliers

,carat,depth,table,price,x,y,z
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
8,0.22,65.1,61.0,337,3.87,3.78,2.49
12,0.22,60.4,61.0,342,3.88,3.84,2.33
14,0.20,60.2,62.0,345,3.79,3.75,2.27
...,...,...,...,...,...,...,...
52993,1.29,67.7,62.0,2596,6.69,6.59,4.50
53064,0.72,56.5,68.0,2608,6.03,5.89,3.37
53540,0.90,72.9,54.0,2691,5.74,5.67,4.16
53542,0.78,54.7,67.0,2691,6.25,6.15,3.40


### belirlediğimiz bir cutoff değeri ile maskeleme yapıyoruz

In [47]:
preds=iforest_model.predict_proba(outliers)

In [58]:
outliers[preds[:,1]>=0.65].index

Int64Index([11182, 11963, 15951, 16283, 17181, 17196, 19346, 19502, 19866,
            21758, 22540, 22741, 22831, 23539, 23644, 24067, 24131, 24297,
            24328, 24394, 24520, 24932, 25850, 25998, 25999, 26123, 26243,
            26431, 26444, 26534, 27112, 27130, 27415, 27429, 27503, 27515,
            27516, 27630, 27638, 27646, 27649, 27679, 27685, 27739, 36503,
            41918, 49189, 49556, 49557, 50773, 52860, 52861],
           dtype='int64')

In [60]:
len(outliers[preds[:,1]>=0.65].index)

52